In [36]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm

In [37]:
class Dictionary(object):
    def __init__(self) -> None:
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
    
    def add_word(self, word) -> None:
        if word not in self.word2idx:        
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __len__(self) -> int:
        return len(self.word2idx)

In [38]:
class TextProcess(object):

    def __init__(self) -> None:
        self.dictionary = Dictionary()
    
    def get_data(self, path, batch_size=20):
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
        
        rep_tensor = torch.LongTensor(tokens) # tensor of length tokens
        index = 0

        # Capture the indices of each word in the tensor for the entire story
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    rep_tensor[index] = self.dictionary.word2idx[word]
                    index += 1
        
        # Divide the data into batches based on batch size 
        num_batches = rep_tensor.shape[0] // batch_size
        rep_tensor = rep_tensor[:num_batches*batch_size]
        rep_tensor = rep_tensor.view(batch_size, -1)
        return rep_tensor


In [39]:
embed_size = 128         # Input features to LSTM. idx -> word embedding size
hidden_size = 1024       # Num of hidden neurons or num of hidden units
num_layers = 2           # 1 layer of LSTM. If 2, 2 LSTM layers stacked on top of each other
num_epochs = 500
batch_size = 20
timesteps = 30           # we look at 30 previous words to predict next word
learning_rate = 0.002    # we can use LR scheduler here

In [40]:
corpus = TextProcess()
rep_tensor = corpus.get_data('17.alice.txt', batch_size)
print(rep_tensor.shape)

torch.Size([20, 1484])


In [41]:
vocab_size = len(corpus.dictionary)
print(vocab)

5290


In [42]:
# batches based on time steps
num_batches = rep_tensor.shape[1] // timesteps
print(num_batches)

49


In [43]:
class TextGenerator(nn.Module):

    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(TextGenerator, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size) # performs the mapping from idx to word embedding
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, h):
        x = self.embed(x)
        out, (h, c) = self.lstm(x, h) # x -> input, h -> initialzed hidden state and memory state
        # Reshape out(batch_size, timesteps, hidden_size) -> (batch_size*timesteps, hidden_size) to be appropriate for FC layer
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        out = self.linear(out)
        return out, (h, c)

In [44]:
model = TextGenerator(vocab_size, embed_size, hidden_size, num_layers)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [45]:
# Train the model
for epoch in range(num_epochs):

    # set initial hidden and cell states
    states = (torch.zeros(num_layers, batch_size, hidden_size),
              torch.zeros(num_layers, batch_size, hidden_size))

    for i in range(0, rep_tensor.size(1) - timesteps, timesteps):
        # Get mini batch inputs and targets
        inputs = rep_tensor[:, i:i+timesteps]
        targets = rep_tensor[:, (i+1):(i+1)+timesteps]

        outputs, _ = model(inputs, states)
        loss = loss_fn(outputs, targets.reshape(-1)) # targets must be in 1 dimension from documentation

        model.zero_grad()
        loss.backward()

        clip_grad_norm(model.parameters(), 0.5) # to prevent exploding gradient problem. 0.5 is max allowed value.
        optimizer.step()

        step = (i+1) // timesteps
        if step % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}")

/tmp/ipykernel_29101/440623552.py:19: FutureWarning: `torch.nn.utils.clip_grad_norm` is now deprecated in favor of `torch.nn.utils.clip_grad_norm_`.
  clip_grad_norm(model.parameters(), 0.5) # to prevent exploding gradient problem. 0.5 is max allowed value.


Epoch [1/500], Loss: 8.5720
Epoch [2/500], Loss: 6.1857
Epoch [3/500], Loss: 5.7727
Epoch [4/500], Loss: 5.4932
Epoch [5/500], Loss: 5.2036
Epoch [6/500], Loss: 4.9250
Epoch [7/500], Loss: 4.5831
Epoch [8/500], Loss: 4.3820
Epoch [9/500], Loss: 4.1414
Epoch [10/500], Loss: 3.9043
Epoch [11/500], Loss: 3.5437
Epoch [12/500], Loss: 3.3074
Epoch [13/500], Loss: 3.0889
Epoch [14/500], Loss: 2.8228
Epoch [15/500], Loss: 2.4328
Epoch [16/500], Loss: 2.1427
Epoch [17/500], Loss: 1.9316
Epoch [18/500], Loss: 1.4732
Epoch [19/500], Loss: 1.1116
Epoch [20/500], Loss: 0.8621
Epoch [21/500], Loss: 0.6765
Epoch [22/500], Loss: 0.4724
Epoch [23/500], Loss: 0.2990
Epoch [24/500], Loss: 0.1922
Epoch [25/500], Loss: 0.1154
Epoch [26/500], Loss: 0.0832
Epoch [27/500], Loss: 0.0705
Epoch [28/500], Loss: 0.0677
Epoch [29/500], Loss: 0.0648
Epoch [30/500], Loss: 0.0634
Epoch [31/500], Loss: 0.0620
Epoch [32/500], Loss: 0.0611
Epoch [33/500], Loss: 0.0602
Epoch [34/500], Loss: 0.0596
Epoch [35/500], Loss: 0

In [46]:
# Test the model - Generate text and see
with torch.no_grad():
    with open('17.results.txt', 'w') as f:
        state = (torch.zeros(num_layers, 1, hidden_size),
              torch.zeros(num_layers, 1, hidden_size))

        input = torch.randint(0, vocab_size, (1,)).long().unsqueeze(1)
    
        for i in range(500):
            output, _ = model(input, state)
            print(output.shape)

            # Sample a word if from the exponential of the output
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()
            print(word_id)

            # Replace input with sampled word id for the next time step
            input.fill_(word_id)

            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print(f"Sample [{i+1}/500] words and save to results.txt")

torch.Size([1, 5290])
16
torch.Size([1, 5290])
55
torch.Size([1, 5290])
5
torch.Size([1, 5290])
103
torch.Size([1, 5290])
93
torch.Size([1, 5290])
1388
torch.Size([1, 5290])
44
torch.Size([1, 5290])
3128
torch.Size([1, 5290])
7
torch.Size([1, 5290])
173
torch.Size([1, 5290])
4451
torch.Size([1, 5290])
3
torch.Size([1, 5290])
2065
torch.Size([1, 5290])
20
torch.Size([1, 5290])
6
torch.Size([1, 5290])
129
torch.Size([1, 5290])
1414
torch.Size([1, 5290])
285
torch.Size([1, 5290])
44
torch.Size([1, 5290])
4441
torch.Size([1, 5290])
34
torch.Size([1, 5290])
55
torch.Size([1, 5290])
3
torch.Size([1, 5290])
3779
torch.Size([1, 5290])
3280
torch.Size([1, 5290])
252
torch.Size([1, 5290])
30
torch.Size([1, 5290])
16
torch.Size([1, 5290])
980
torch.Size([1, 5290])
4789
torch.Size([1, 5290])
103
torch.Size([1, 5290])
3307
torch.Size([1, 5290])
5
torch.Size([1, 5290])
439
torch.Size([1, 5290])
209
torch.Size([1, 5290])
3
torch.Size([1, 5290])
915
torch.Size([1, 5290])
4355
torch.Size([1, 5290])
34
